<a href="https://colab.research.google.com/github/SanderSaska/MorfoloogiliseMuuttyybiAutomaatneTuvastaja/blob/main/Morfoloogilise_muutt%C3%BC%C3%BCbi_automaatne_tuvastamine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Colabi jaoks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Lõputöö')

Mounted at /content/drive


## Statistika

In [10]:
with open('fs_lex.txt', 'r') as fs_lex:
  # Leiame ridade arvu
  ridade_arv = 0
  for rida in fs_lex:
    ridade_arv += 1
  print(ridade_arv) # 73832

73832
